In [ ]:
import papermill as pm
import pandas as pd
from multiprocessing import Pool
import concurrent.futures
import queue
import os
from pathlib import Path
from datetime import datetime

In [ ]:
def run_notebook(parameters,notebook_to_run,parameters_common={}):
    
    # change to directory where the notebook is (resolve relative imports)
    os.chdir(Path(notebook_to_run).absolute().parent)
    
    # run notebook
    for parameters_spec in parameters_list:
        parameters = {**parameters_common, **parameters_spec}

        pm.execute_notebook(
            notebook_to_run,
            '/dev/null',
            parameters=parameters)

# 0) Create projections
**In:** folder with `.nd2` images

**Out:** folder `projections` with maximum inensity projections `.png` images split by channel

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/raw"}
]
notebook_to_run = "/home/stumberger/ep2024/subscripts/nd2_make_projections.ipynb"

run_notebook(parameters_list,notebook_to_run)

# 1) Resave .nd2 to tif 
**In:** folder containing folder `raw` with `.nd2` images

**Out:** folder `tif` with `.tif` images split by channel

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/resave_nd2_as_tif.ipynb"

run_notebook(parameters_list,notebook_to_run)

# 2) Spot detection

**In:** 
- `tif_subfolder` containing single channel `tif` images
- `out_subfolder` containing spot detection settings file `chX.txt` generated via RS-FISH Fiji plugin for each channel specified in `channels`
- `acquisition_info.json` file with the sample metadata which you wish to be added to your spots

**Out:**
- folder containing a `csv` file for each provioded `tif`
- `merge.csv` file combining spots and their metadata from all images in the batch
- vis folder with detection visualization for all `tif`s as `png` (good for checking detection results)

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"}
]

parameters_common = {"channels": [1,2],
                    "tif_subfolder": "tif",
                    "out_subfolder": "detections"}

notebook_to_run = "/home/stumberger/ep2024/subscripts/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 3) Correct chromatic shift
**In:** 
- `.json` chromatic shift estimation created with `chromatic_aberration_estimation_elastix.ipynb`
- `.csv` table containing original coordinates and channels

**Out:**
- `.csv` tables with additional columns containing the corrected coordinates and the reference channel

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/detections/"}
]

parameters_common = {"pixel_size": [0.3,0.13,0.13],
                     "channel_aliases" : {
                        '405-CSU-W1': '405-CSU-W1',
                        '488-CSU-W1': '488-CSU-W1',
                        '561-CSU-W1': 1,
                        '640-CSU-W1': 2},
                     "reference_channel": '405-CSU-W1',
                     "transforms_path": "/home/stumberger/ep2024/example/chrom_shift_reference/nups_channel_registration.json"
                    }

notebook_to_run = "/home/stumberger/ep2024/subscripts/correct_chromatic_aberration_for_tables.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 4) Segment cells
**In:**
- `in_path` folder containing `tif` folder with single channel `.tif` images
- cellpose model

**Out:**
- `segmentation` folder with segmentation masks
- `segmentation/vis` subfolder with segmentation visualizations


In [ ]:
import papermill as pm
import concurrent.futures
import queue

def execute_notebook(parameters):
    pm.execute_notebook(
        '/home/stumberger/ep2024/subscripts/cellpose_segmentation_3d.ipynb',
        None,
        parameters=parameters)

def process_parameters(parameters_queue, parameters_common):
    while not parameters_queue.empty():
        parameters_spec = parameters_queue.get()
        execute_notebook({**parameters_common, **parameters_spec})
        parameters_queue.task_done()

if __name__ == '__main__':
    parameters_common = {
        "model": "/home/stumberger/ep2024/example/segmentation_model/es_20231026"}

    parameters_list =     parameters_list = parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"}
]

    batch_size = 3  # Number of parallel tasks

    parameters_queue = queue.Queue()
    for parameters_spec in parameters_list:
        parameters_queue.put(parameters_spec)

    with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
        # Start the initial batch
        initial_batch = [executor.submit(execute_notebook, {**parameters_common, **parameters_queue.get()}) for _ in range(batch_size)]
        
        # Continue processing new tasks as previous ones complete
        while not parameters_queue.empty():
            # Wait for any task in the initial batch to complete
            concurrent.futures.wait(initial_batch, return_when=concurrent.futures.FIRST_COMPLETED)
            
            # Replace completed tasks with new tasks
            completed = [task for task in initial_batch if task.done()]
            for task in completed:
                initial_batch.remove(task)
                new_task = executor.submit(execute_notebook, {**parameters_common, **parameters_queue.get()})
                initial_batch.append(new_task)

        # Wait for all tasks to complete
        concurrent.futures.wait(initial_batch)

    parameters_queue.join()

# 4.1) Add segmentation info to spots
**In:**
- segmentation masks in either `npy` or `png` format (name should correspond to the name of the image in the spot file)
- `.csv` file of spots in each of the 2 channels, containg following columns:
    - *img* - image name
    - *channel* channel number the spot belongs to  
    - *x, y* and *z* coordiantes

**Out:**
- `.csv` file of spots with addtional columns:
    - *cell* - cell number in that particular image (0 means outside of cell)
    - *whole_cell* - marks `False` for all cells touching the image border
- optionally removes spots outside of cells

In [ ]:
parameters_common = {
    "filter": True, # remove spots outsde of cells?
    "mask_ending": "_seg",
    "spot_file": "merge_shift-corrected.csv"}

parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/assign_spots_to_cell.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 4.2) Quality measure: calculate sensitivity
**In:**
- segmentation masks in either `npy` or `png` format (name should correspond to the name of the image in the spot file)
- `.csv` file of spots containing:
    - *img* - image name
    - *channel* - channel number the spot belongs to  
    - *x, y* and *z* coordiantes

**Out:**
- `spots_per_cell.csv` with information on how many spots each cell in an image and channel contains.

In [ ]:
parameters_common = {"mask_ending": "_seg",
                    "rel_spot_path": "/detections/merge_filtered.csv"} #spot path relative to upper folder}

parameters_list = [
    {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/calculate_spots_per_cell.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 5) Calculate distances
**In:** 
- `in_path` working folder
- `rel_spot_path` subpath to the processed spot files, relative to `in_path`

**Out:**
- `distances.csv` file with distances between spots matched in 2 channels

In [ ]:
parameters_common = {
    "rel_spot_path": "/detections/merge_shift-corrected.csv",
        "channels": [1,2],
        "voxel_size": [300, 150, 150]}

parameters_list = [
     {"in_path": "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/paired_spot_distances_2_channels.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 6) Join csvs (optional)
**In:** Paths to multiples folders containing the `distances.csv` files you want to join

**Out:** `all_distances_filtered_{time}.csv` file with all `.csv`s joined

In [ ]:
wd = "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"

csv_files = [
    "/home/stumberger/ep2024/spinning_disk_spot_detection/example/"
    ]

# Initialize an empty list to store DataFrames
dataframes = []

# Read and store each CSV file as a DataFrame
for file in csv_files:
    df = pd.read_csv(f"{file}/distances.csv")
    dataframes.append(df)

# Join the DataFrames using Pandas (e.g., concatenate them vertically)
joined_dataframe = pd.concat(dataframes, ignore_index=True)

# Save the joined DataFrame to a new csvfile
time =  datetime.now().strftime("%Y-%m-%d-%H-%M")
joined_dataframe.to_csv(f'{wd}/all_distances_filtered_{time}.csv', index=False)